In [2]:
import pandas as pd
import os
import sys
os.getcwd()
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
# #feature extraction using VGG-19
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot

In [3]:
from tqdm import tqdm
#print(tf.__version__)
train = pd.read_csv('C:\\Users\\sarita.mukherjee\\Downloads\\Capstone_Project\\train.csv')
fashion_dir= "C:\\Users\\sarita.mukherjee\\Downloads\\Capstone_Project\\fashion\\"
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(fashion_dir+train['images'][i], target_size=(224,224,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
#coverting the images to array
images = np.array(train_image)
images

100%|█████████████████████████████████████████████████████████████████████████████| 6000/6000 [00:25<00:00, 235.65it/s]


array([[[[0.7529412 , 0.5058824 , 0.36862746],
         [0.7490196 , 0.5019608 , 0.3647059 ],
         [0.73333335, 0.4862745 , 0.34901962],
         ...,
         [0.9098039 , 0.9098039 , 0.9098039 ],
         [0.9098039 , 0.9098039 , 0.9098039 ],
         [0.9098039 , 0.9098039 , 0.9098039 ]],

        [[0.74509805, 0.49803922, 0.36078432],
         [0.74509805, 0.49803922, 0.36078432],
         [0.73333335, 0.4862745 , 0.34901962],
         ...,
         [0.9098039 , 0.9098039 , 0.9098039 ],
         [0.9098039 , 0.9098039 , 0.9098039 ],
         [0.9098039 , 0.9098039 , 0.9098039 ]],

        [[0.75686276, 0.50980395, 0.37254903],
         [0.74509805, 0.49803922, 0.36078432],
         [0.7372549 , 0.49019608, 0.3529412 ],
         ...,
         [0.9098039 , 0.9098039 , 0.9098039 ],
         [0.9098039 , 0.9098039 , 0.9098039 ],
         [0.9098039 , 0.9098039 , 0.9098039 ]],

        ...,

        [[0.9098039 , 0.9098039 , 0.9098039 ],
         [0.9098039 , 0.9098039 , 0.9098039 ]

In [4]:
model= VGG16(include_top=False,pooling='avg',input_shape=(224,224,3))
features = model.predict(images)
#features

array([[6.58534050e-01, 0.00000000e+00, 3.31872404e-02, ...,
        9.66904685e-03, 5.75882852e-01, 0.00000000e+00],
       [3.88082862e-02, 0.00000000e+00, 4.78922098e-04, ...,
        0.00000000e+00, 3.49031180e-01, 0.00000000e+00],
       [1.30553424e-01, 0.00000000e+00, 1.12434831e-02, ...,
        0.00000000e+00, 4.96104002e-01, 0.00000000e+00],
       ...,
       [4.19535458e-01, 0.00000000e+00, 2.80847456e-02, ...,
        1.40647385e-02, 3.81256968e-01, 0.00000000e+00],
       [1.11042783e-01, 0.00000000e+00, 9.04268306e-03, ...,
        7.35880211e-02, 7.78933764e-01, 0.00000000e+00],
       [1.84851438e-01, 0.00000000e+00, 3.19072157e-01, ...,
        0.00000000e+00, 6.22527063e-01, 0.00000000e+00]], dtype=float32)

In [15]:
features=pd.DataFrame(features)
features.head()


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.658534,0.0,0.033187,0.004680,0.040309,0.0,0.011067,0.071000,0.0,0.194333,...,1.663284,0.373922,0.293564,0.002317,0.001852,0.492725,0.015107,0.009669,0.575883,0.0
1,0.038808,0.0,0.000479,0.000000,0.101650,0.0,0.036146,0.072419,0.0,0.000000,...,1.397959,0.055577,0.167094,0.035621,0.000000,0.126570,0.009334,0.000000,0.349031,0.0
2,0.130553,0.0,0.011243,0.000000,0.168639,0.0,0.242627,0.011844,0.0,0.000000,...,1.258340,0.059814,0.313998,0.004248,0.000859,0.203299,0.001681,0.000000,0.496104,0.0
3,0.035573,0.0,0.024319,0.314908,0.082578,0.0,0.000000,0.000000,0.0,0.106918,...,1.292763,0.052405,0.229397,0.052640,0.009312,0.595823,0.008690,0.000000,0.486992,0.0
4,0.177025,0.0,0.053214,0.000000,0.123413,0.0,0.117448,0.031517,0.0,0.000000,...,1.349733,0.503748,0.416155,0.184453,0.000057,0.210542,0.251382,0.002283,0.342213,0.0


In [18]:
#labelling the features
label=['shoes','beauty','jewelry and watches','bags']
features['target']= label
features
# X=features.drop('target',axis=1)
# y=features['target']
# #Label encoder
# from sklearn.preprocessing import LabelEncoder
# enc=LabelEncoder()
# y=enc.fit_transform(y)

ValueError: Length of values (4) does not match length of index (6000)